Part I: 
- runs all loading of the .dump files
- divides the loaded data into chunks: one chunk for each snapshot
- calculates the computationally heavy calculation of disp, N_N, d_5NN etc
- saves the dataframes with all required features into the features_csv folder

NB: All clustering algorithm calculation will be done in Part II


# Clustering Algorithm applied to atom classification

- Clustering algorithm used to classify atoms as crystal-like and glass-like based on the features
- Clustering applied to each snapshot
- Program must be run from the directory that contains individual directory for each snapshot
- Each directory must be named in a pre-defined fashion (0000ps, 0100ps, 0200ps,............., 2000ps)

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

- *.dump file contains the dump obtained from the lammps output (Has all the LAMMPS calculated quantities for each atom)
- LAMMPS does not arrange the atoms in order of their indices like DL-POLY does

In [2]:
import glob, os

def load_dump_files(file_path, label):
    '''Inputs:
        file_path=path to the directories where the dump files are located
        label = ID for the atom to be pulled out from the LAMMPS dump file
       Outputs:
        dump_contents = contents of variable values form all the snapshots in chronological order
        num_snaps = total number of snapshots present in the study       
        box = list of the box size at each snapshot
    '''
    dump_contents, box = [],[]
    files = sorted(glob.glob(file_path, recursive=True)) # sorts dump files in chronological order
    num_snaps = 0
    for filename in files:
        num_snaps += 1
        with open(filename, 'r',encoding='utf-8') as infile:
            print(filename)
            txt = infile.readlines()
            # Skip first 9 lines that contains system information not coordinates
            for line in txt[9:]:                        
                if line.split()[1] == label: 
                    dump_contents.append(np.array(line.split()).astype(float) )
            box.append(float(txt[5].split()[1]) - float(txt[5].split()[0]))
    return dump_contents, num_snaps, box


In [3]:
# label="3" for O, "2" for Nb, "1" for Li
# The description of crystallization based on the oxygen network and order is unique in the study of nucleation.
dump_contents, num_snaps, box = load_dump_files(file_path = './**.dump', label="3")

./0000ps.dump
./0050ps.dump
./0100ps.dump
./0150ps.dump
./0200ps.dump
./0250ps.dump
./0300ps.dump
./0350ps.dump
./0400ps.dump
./0450ps.dump
./0500ps.dump
./0550ps.dump
./0600ps.dump
./0650ps.dump
./0700ps.dump
./0750ps.dump
./0800ps.dump
./0850ps.dump
./0900ps.dump
./0950ps.dump
./1000ps.dump
./1050ps.dump
./1100ps.dump
./1150ps.dump
./1200ps.dump
./1250ps.dump
./1300ps.dump
./1350ps.dump
./1400ps.dump
./1450ps.dump
./1500ps.dump
./1550ps.dump
./1600ps.dump
./1650ps.dump
./1700ps.dump
./1750ps.dump
./1800ps.dump
./1850ps.dump
./1900ps.dump
./1950ps.dump
./2000ps.dump
./2050ps.dump
./2100ps.dump
./2150ps.dump
./2200ps.dump
./2250ps.dump
./2300ps.dump
./2350ps.dump
./2400ps.dump
./2450ps.dump
./2500ps.dump
./2550ps.dump
./2600ps.dump
./2650ps.dump
./2700ps.dump
./2750ps.dump
./2800ps.dump
./2850ps.dump
./2900ps.dump
./2950ps.dump


In [4]:
print(f"Total number of snapshots = {num_snaps}")
print(f"Total simlations time = {(num_snaps-1)*0.05} ns")

NAT=(int(len(dump_contents)/num_snaps))
print("No. of atoms: %d \nNo. of snaps: %d"%(NAT, num_snaps))

# Convert the dump_contents list into a dataframe and name the columns
df = pd.DataFrame(dump_contents)

df.columns=['ID',"Atom","x","y","z","Q4","Q6","Q8","Q10","Nc_6","Nc_8","c_x","c_y","c_z"]
df.describe

Total number of snapshots = 60
Total simlations time = 2.95 ns
No. of atoms: 299850 
No. of snaps: 60


<bound method NDFrame.describe of                 ID  Atom          x          y          z        Q4        Q6  \
0         298084.0   3.0    4.69560    2.07947    3.47650  0.257916  0.263582   
1         444777.0   3.0    5.43723    3.83086    1.12886  0.199310  0.285326   
2         350567.0   3.0    3.49642    4.64635    2.97165  0.258112  0.252274   
3         304599.0   3.0    5.20663    4.38183    5.10384  0.111567  0.195211   
4         474003.0   3.0    2.01913    4.61225    5.37254  0.142438  0.295282   
...            ...   ...        ...        ...        ...       ...       ...   
17990995  464139.0   3.0  182.67400  180.12300  182.93900  0.153692  0.343750   
17990996  209404.0   3.0  182.45100  184.36300  183.90200  0.169440  0.229342   
17990997  425718.0   3.0  179.86500  183.18600  183.55900  0.235634  0.243449   
17990998  334491.0   3.0  184.82100  183.40400  185.84800  0.217409  0.269662   
17990999  350197.0   3.0  181.53500  182.29500  181.51000  0.220736  0.2605

In [5]:
# Change the data type of the columns appropriately
df = df.astype({"ID": int, "Atom": int, "Nc_6":int, "Nc_8":int})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17991000 entries, 0 to 17990999
Data columns (total 14 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID      int64  
 1   Atom    int64  
 2   x       float64
 3   y       float64
 4   z       float64
 5   Q4      float64
 6   Q6      float64
 7   Q8      float64
 8   Q10     float64
 9   Nc_6    int64  
 10  Nc_8    int64  
 11  c_x     float64
 12  c_y     float64
 13  c_z     float64
dtypes: float64(10), int64(4)
memory usage: 1.9 GB


### Feature Engineering begins here

- Keep only columns that will be needed later

In [6]:
df = df[['ID','x','y','z','Q6','Nc_6']]
df.describe()

,ID,x,y,z,Q6,Nc_6
count,1.799100e+07,1.799100e+07,1.799100e+07,1.799100e+07,1.799100e+07,1.799100e+07
mean,3.498255e+05,9.086531e+01,9.086255e+01,9.086378e+01,2.695268e-01,3.854307e+00
std,8.655924e+04,5.405278e+01,5.404763e+01,5.404562e+01,4.569666e-02,5.706098e+00
min,1.999010e+05,-4.453660e+00,-4.452970e+00,-4.453350e+00,4.628170e-02,0.000000e+00
25%,2.748630e+05,4.410970e+01,4.411020e+01,4.413217e+01,2.384730e-01,1.000000e+00
50%,3.498255e+05,9.085140e+01,9.086250e+01,9.087030e+01,2.709170e-01,2.000000e+00
75%,4.247880e+05,1.376260e+02,1.376100e+02,1.376160e+02,3.022190e-01,4.000000e+00
max,4.997500e+05,1.861910e+02,1.861910e+02,1.861900e+02,4.924310e-01,4.200000e+01


__Looks exactly the way we want our data to be!__

- To perform clustering on each snapshot, we need to divide the dataframe into chunks for each snapshot
- Each chunk must have NAT atoms
- Each chunk needs to be sorted because the LAMMPS dump does not sort by ID

In [7]:
split_dataframes = [df[i:i+NAT] for i in range(0, len(df), NAT)]
sorted_chunks = [chunk.sort_values(by='ID') for chunk in split_dataframes]

# If you want to see what the chunks look like, uncomment the lines below
# for i, split_df in enumerate(sorted_chunks):
#     print(f"DataFrame {i + 1}:")
#     print(split_df)
#     print()

sorted_chunks[0].tail(20)


,ID,x,y,z,Q6,Nc_6
296000,499731,177.72700,147.8840,169.7990,0.287412,3
271636,499732,97.50660,114.8020,115.6060,0.327409,4
270821,499733,159.03900,174.5070,107.1800,0.321868,2
266550,499734,134.37600,132.0620,102.3350,0.320092,4
119666,499735,10.59080,138.7910,110.3060,0.322617,1
257283,499736,106.92800,114.3240,78.2849,0.366338,2
199572,499737,156.37200,14.2575,119.6440,0.260117,6
270232,499738,158.71800,153.0710,109.9430,0.342578,1
298532,499739,100.46200,141.4660,179.7810,0.268292,4
260837,499740,157.59500,132.2040,87.7735,0.353826,1


## One more feature (Displacement)


### __How do we calculate the displacement feature?__   
- For each snapshot, calculate the disp for each atom at four different 5ps snaps just before or after the snap
- For e.g, the disp for 200ps is calculated as the average of the disp at 195 ps, 190ps, 185ps, 180ps
- disp at 195: dist(r_195ps, r_190ps)
- We need to pull out the coordinates from the HISTORY file to do so

In [8]:
def periodic_distance(atom, nbd, box):
    '''Inputs:
        atom = coordinate of a single reference atom
        nbd = array of coordinates of multiple atoms
        box = length of the box
       Outputs:
        dis = distance between atom and nbd
        len(dis) == len(nbd)
    '''
    delta = np.abs(atom-nbd)
    delta = np.where(delta> 0.5*box, delta-box, delta)
    dis = np.sqrt((delta**2).sum(axis=-1))
    return dis

In [9]:
for ind in range(1, num_snaps):
    current = sorted_chunks[ind].iloc[:, 1:4].to_numpy()
    previous = sorted_chunks[ind-1].iloc[:, 1:4].to_numpy()
    
    dist = periodic_distance(current, previous, box[ind])
    sorted_chunks[ind]['disp'] = dist

In [10]:
sorted_chunks[-1].describe()

,ID,x,y,z,Q6,Nc_6,disp
count,299850.000000,299850.000000,299850.000000,299850.000000,299850.000000,299850.000000,299850.000000
mean,349825.500000,90.829524,90.834612,90.886317,0.272491,6.901434,4.713582
std,86559.383446,54.873876,54.874964,54.885754,0.045286,9.077699,3.351843
min,199901.000000,-4.453660,-4.452970,-4.453350,0.071516,0.000000,0.013990
25%,274863.250000,43.466775,43.461050,43.510200,0.241152,1.000000,1.861932
50%,349825.500000,90.770650,90.803150,90.928850,0.276726,3.000000,4.613717
75%,424787.750000,138.215000,138.189500,138.292750,0.306538,6.000000,6.946417
max,499750.000000,186.191000,186.191000,186.190000,0.456012,40.000000,27.569397


- Great the disp features has been added to 100ps, 200ps, .............., 1700ps
## More features (d_5NN, N_N)


### __How do we calculate these features?__   
- __d_5NN__: average distance of 5 nearest neighbors with Nc > 10 
- __N_N__: number of atoms with 5A that have Nc > 10

__d_5NN and N_N calculator using multiprocessing__

In [11]:
import multiprocessing
print("Number of CPU cores available: ",multiprocessing.cpu_count())


Number of CPU cores available:  128


In [12]:
import multiprocessing
from tqdm import tqdm

def process_snapshot(args):
    df_snap, box = args
    num_neigh_list, avg_neigh_dist = [], []
    for j in tqdm(range(len(df_snap)), desc="Processing"):
        filtered_particles = df_snap[df_snap['Nc_6'] > 10]
        particle_row = df_snap.iloc[j]
        coordinates = [particle_row['x'], particle_row['y'], particle_row['z']]
        distances = periodic_distance(coordinates, filtered_particles[['x','y','z']], box)
        # Count neighbors within the distance threshold
        num_neigh = np.sum(distances < 5.0)
        if min(distances) == 0:
            num_neigh -= 1  # Exclude self
        num_neigh_list.append(num_neigh)
    return num_neigh_list 

def process_snapshots(sorted_chunks, box):
    num_snaps = len(sorted_chunks)
    # creates a multiprocessing pool object,
    # which allows you to execute functions concurrently across multiple CPUs.
    pool = multiprocessing.Pool()
    results = pool.map(process_snapshot, [(df_snap, box[ind]) for ind, df_snap in enumerate(sorted_chunks)])
    pool.close()
    pool.join()
    return results

results = process_snapshots(sorted_chunks, box)

# Add the results back to the dataframe chunks
for ind, (num_neigh_list) in enumerate(results):
    sorted_chunks[ind]['N_N'] = num_neigh_list


Processing:  27%|██▋       | 79979/299850 [03:23<09:20, 392.55it/s]]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Processing:  14%|█▍        | 41366/299850 [03:48<24:34, 175.30it/s]]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Processing:  35%|███▍      | 104148/299850 [04:15<06:27, 505.66it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the 

In [13]:
sorted_chunks[-1]

,ID,x,y,z,Q6,Nc_6,disp,N_N
17795788,199901,60.9147,113.1610,69.2511,0.325255,32,0.368619,20
17792833,199902,60.9172,111.2600,66.9531,0.302210,23,0.372973,24
17792636,199903,59.2457,108.9290,67.0710,0.312688,16,0.439988,22
17795787,199904,60.6776,111.5690,74.0545,0.315932,24,0.386312,22
17795786,199905,59.3186,112.3890,71.6224,0.327186,22,0.521044,21
...,...,...,...,...,...,...,...,...
17969049,499746,127.3590,125.8460,132.9800,0.270126,3,2.191484,7
17973539,499747,177.5820,177.4480,138.7930,0.302307,3,9.655227,0
17977565,499748,151.3500,117.9010,151.8340,0.182791,1,6.490519,0
17986156,499749,139.0210,118.9710,171.7880,0.292113,3,8.230012,0


In [14]:
import pandas as pd
import os

# Create a directory to store CSV files
output_directory = "features_csv"
os.makedirs(output_directory, exist_ok=True)

# Loop through the DataFrames and save them as CSV files
for ind in range(1, num_snaps):
    # Define the file name (you can customize this)
    file_name = os.path.join(output_directory, f"{ind*50}ps.csv")
    
    # Save the DataFrame as a CSV file
    sorted_chunks[ind].to_csv(file_name, index=False)

print("CSV files saved successfully.")


CSV files saved successfully.


__Let us look at how our variables are correlated__


In [17]:
pwd


'/ocean/projects/mat230020p/rt887917/LiNbO3_LAMMPS/Multiple_Seeds/500k/Seed_Sep_55A/KMeans_Clustering/Z0'

import seaborn as sns
import matplotlib.pyplot as plt

NN_disp_corr, NN_Nc_corr, NN_dist_corr = [],[],[]
for ind in range(1,num_snaps):
    data = sorted_chunks[ind][['Q6','Nc_6','disp','N_N','dist_from_c']]
    correlation_matrix = data.corr()
    NN_dist_corr.append(correlation_matrix['N_N'].iloc[4])
    NN_disp_corr.append(correlation_matrix['N_N'].iloc[2])
    NN_Nc_corr.append(correlation_matrix['N_N'].iloc[1])
# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
plt.plot(NN_disp_corr, label="NN-disp")
plt.plot(NN_dist_corr, label="NN-dist")
# plt.plot(NN_Nc_corr, label="NN-Nc")
plt.legend()
# plt.savefig("conf_mat.png",dpi=500,bbox_inches='tight')
plt.show()


# Create a heatmap of the correlation matrix in the last snapshot
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.savefig("conf_mat.png",dpi=500,bbox_inches='tight')
plt.show()
